# Juliaで数値計算 浸透流にトライ （つくりかけ）

## 対象読者
1. Juliaで数値計算に興味がある人
2. 浸透流の計算に興味がある人
3. 自分。

浸透流、やってみたいけど全くわからん。

これが比較的わかりやすいので参考にした。
https://inis.iaea.org/collection/NCLCollectionStore/_Public/31/027/31027904.pdf

一番わからん貯留係数はやっぱり複雑だった。  
https://www.jstage.jst.go.jp/article/jjseg1960/39/5/39_5_475/_pdf



## 基礎方程式 （リチャーズの式）
yが鉛直重力反対方向（上向き）で２次元の多孔質中の流れを考える。最終的に計算したいイメージは乾いた珪藻土マットの上にスポンジがおいてあって、スポンジの中央上から水を流し込む状態。最初は乾いた状態に水が来るので、連続の式はゼロにならないわ、2種類の材料があるので透水係数が均質じゃないわで、ラプラス方程式を解くだけの拡散方程式に帰着できないのがむずかしいところ。

### 連続の式（みたいなやつ）
いきなりここで引っかかるが、単位セルあたりの水の体積Q（この業界では$\theta$が普通）を考えてみると理解できた。つまり、単位時間あたりのセルの水の量の変化$\frac{\partial Q}{\partial t}$が単位流速のセルに流出入する流量と等しいので（１）になる（[広義の連続の式の導出](https://ja.wikipedia.org/wiki/連続の方程式)とおなじ）が、たぶんこのままだと使いにくいので、$S_s = \frac{\partial Q}{\partial h}$なる比貯留係数なるものを使って書き直してる（２）式。結局、貯留係数はに単位水頭の減少がおきたときにに単位セルから排出される流体の体積ってことらしい。うーん、わからん。まず[水頭の概念](https://ja.wikipedia.org/wiki/水頭)が非常にわかりにくい…。

$$
\frac{\partial Q}{\partial t} = -\frac{\partial u_x}{\partial x} -\frac{\partial u_y}{\partial y} \tag{1}
$$
$$
S_s \frac{\partial h}{\partial t} = -\frac{\partial u_x}{\partial x} -\frac{\partial u_y}{\partial y} \tag{2}
$$
$$
t:時間 (s)，h:圧力水頭 (cm)，u_x u_y，:x， y方向のダルシ一流速 (cm/s), S_s :比貯留係数
$$
圧力水頭はよくわかってないんだけども、水の圧力を考える場合には、高さに直すのが非常にわかりやすいので地質学（？）の分野ではよく導入されている概念。1cm水が静止状態で積みあがってたらその下には1gfの力がかかっててこれが水圧になるのでこれが高さ水頭（ｃｍ）の考え（のはず）。で、それと同じ圧力の圧力（ｃｍ）が圧力水頭ということのはず（間違ってたらコメントください）  

ダルシー流速はみかけの流速。つまり、単位セルの境界上での見かけの速さ。何を言っているかというと多孔質層の内部では固体でない部分をくねくね流体がながれているため、実際の速度と、セル全体としてみた見かけの流速と真の流速は違うって話。ここでは真の流速には用はないので気にせず流速といったらみかけの流速で。

なお、通常は連続の式は左辺が0になっている形式を使うけど、今回は最初にかいてあるように、湧き出しがあるに等しい状態なのでゼロにならない。

### ダルシー則
多孔質層中を流れる流体の圧力損失は透水係数に比例するという経験式。最近はちゃんとナビエストークスの式とかと関連付けて確認されているらしい。レイノルズ数が十分に低ければだいたい成り立つらしい（スポンジの多孔質が十分細かくないとなりたたないってことかな？）

x方向とz方向とそれぞれ考える。透水係数は本当はx方向とy方向で違ったりするらしいので一応わけておく。
$$
u_x = -k_x \frac{\partial h}{\partial x} \tag{3}
$$
$$
u_y = -k_y \left( \frac{\partial h}{\partial y} + 1 \right) \tag{4}
$$

（４）式にはまったので一応補足。流体のすべてのエネルギー（全水頭）は圧力エネルギー（圧力水頭）、運動エネルギー（速度水頭）、位置エネルギー（位置水頭＝高さ水頭）のうち、ダルシー則が成り立つ程度の遅い流速では速度水頭は無視できるほど小さい。で、yは鉛直方向なので位置エネルギーが考慮される。  更に今は水頭表記（cm単位）なので、水の高さyに流体の密度をかけて、水の密度で割ったものがそのまま位置水頭になる。（つまり、流体の質量に重力加速度と高さかけたものが位置エネルギーだけど、それを基準にした単位なので高さだけでOK)  今は流体を水なので密度も無視して高さだけでいい。

$$
\phi = h + y \tag{5}
$$
$$
\frac{\partial \phi}{\partial y} = \frac{\partial (h+y)}{\partial y} = \frac{\partial h}{\partial y} + \frac{\partial y}{\partial y} = \frac{\partial h}{\partial y} + 1 \tag{6}
$$
ってことで、全水頭$\phi$を考えてyで微分すると1がわいてきたので、こいつが重力を表しているっぽい。（変形あってるよね？　[wikipedia:リチャーズの式](https://ja.wikipedia.org/wiki/リチャーズ式)



### 離散化
格子の絵をスマートに書く方法がわからない…。[こちらの記事](https://qiita.com/KQTS/items/9d2a92ef4046d80ef4a5)のコロケート格子でセルの中心の圧力を考えます。
1回微分がおるので、差分とるのに1/2なる概念（iとi+1の平均）がでてきて格子の辺を1/2とみてるはずで、これはコロケート格子なはず…。

まずは(2)式の差分をとる。左辺は時間差分を前進でとって右辺は場所の差分を中央差分でとる（であってる？）。
$$
S_s \frac{h^{n+1}_{i,j}-h^{n}_{i,j}}{\Delta t} = - \frac{u_{x_{i+1/2,j}}^{n+1} - u_{x_{i-1/2,j}}^{n+1}}{\Delta x} - \frac{u_{y_{i,j+1/2}}^{n+1} - u_{y_{i,j-1/2}}^{n+1}}{\Delta y} \tag{7} \\
\Delta t : 差分時間間隔, \Delta x, \Delta y :x,yそれぞれの差分格子間隔（セル間の間隔）,n: 時間ステップ, i, j: x, yそれぞれの格子（セル）点
$$

これを(3),(4)に代入して$h^{n+1}$で整理すると次の式になるらしい（めんどくさいので自分で検算していない←  
a～dにまとめるのみやすくていいなー

$$
h^{n+1}_{i,j} = \frac{b^{n+1}\Delta t +c^{n+1}\Delta t + d^{n+1/2}\Delta t + S_s h^n_{i,j}}{a^{n+1}\Delta t +S_s} \tag{8}
$$
$$
a^{n+1} = \frac{k_{x,i-1/2,j}+k_{x,i+1/2,j}}{\Delta x^2} + \frac{k_{y,i,j-1/2}+k_{y,i,j+1/2}}{\Delta y^2} \\
b^{n+1} = \frac{k_{x_{i-1/2,j}}h^{n+1}_{i-1,j}+k_{x_{i+1/2,j}}h^{n+1}_{i+1,j}}{\Delta x^2}\\
c^{n+1} = \frac{k_{y_{i,j-1/2}}h^{n+1}_{i,j-1}+k_{y_{i,j+1/2}}h^{n+1}_{i,j+1}}{\Delta y^2}\\
d^{n+1/2} = \frac{k_{y_{i,j+1/2}}-k_{y_{i,j-1/2}}}{\Delta y} \tag{9}
$$

元の時間差分が完全陰解法になってるっぽいですね。ホントは[ここ](https://qiita.com/KQTS/items/0c4f6c47a4d56881a178)でいうλが0.75くらいがいいらしい。どこでみたかたどれない…。

で、緩和係数$\omega$を使った[SOR法(これがわかりやすかった)](https://qiita.com/sci_Haru/items/b98791f232b93690a6c3#%E8%A3%9C%E9%81%BA2-%E6%95%B0%E5%80%A4%E8%A7%A3%E6%B3%95-sor%E6%B3%95--%E3%83%A4%E3%82%B3%E3%83%93%E6%B3%95%E3%81%AE%E8%8B%A5%E5%B9%B2%E3%81%AE%E6%94%B9%E5%96%84-)でとくことになってる。
$$
h^{n+1,m+1}_{i,j} = h^{n+1,m}_{i,j} + \omega \left( h'^{n+1,m+1}_{i,j} - h^{n+1,m}_{i,j} \right) \tag{9}
$$

で、差が適当な閾値如何になったら終わり、と。$u_x,u_y$は(2),(3)の差分式で求める。

実際問題、$S_s$は定数ではないっぽいけども、~~定数としておいてやらないと解けるものも解けないの定数として（最初しなくて詰んだ）解いてみる。(やっぱないと無理かも？）~~
結局、不飽和（水がかけてる領域）を扱う以上、何かしら圧力と含水率の関係が定式化してないととけなくて、微分したらでてきちゃうので、それをつかう。今回は、Wikipediaにのってた式でいってみる。上の離散化の式もこれだとおかしいね。$S_{s_{i,j}}^{n+1}$で読み替えて許して…。

Wikipedia:[水分保持曲線](https://ja.wikipedia.org/wiki/水分保持曲線)より
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/215d9c2dda9d084a016a5a86296300f917923472">
を使用。まったくよくわからないし、いろいろあるらしい。

In [108]:
dx = 0.1 # Δx(cm)
dy = 0.1 # Δy(cm)
LLx = 10 #領域の大きさx方向(cm)
LLy = 10 #領域の大きさy方向(cm)

Lx = round(Int, LLx / dx) #セル数
Ly = round(Int, LLy /dy)　#セル数

Qmax = 0.5 #普通はθs
Qmin = 0.0 #普通はθr

#初期化
h = ones(Lx, Ly) * 10.0 #水がないほうが圧力が高いっぽい式になってたので、適当な高めの値を初期値にいれた。よくわからない。
Q = zeros(Lx, Ly)
ux = zeros(Lx+1, Ly) #uはセルとセルの辺上にある。(1,j)が(-1/2,j)
uy = zeros(Lx, Ly+1) #uはセルとセルの辺上にある。(i,1)が(i,-1/2)

k = ones(Lx+1,Ly+1)*0.01 #kもセルとセルの辺上にある。

for j in 51:Ly+1
    k[:,j] = k[:,j]*10 #上の方にあるスポンジ層は下の方にある層の透水係数に対して10倍
end
    
for i in 40:61
    Q[i,end] = Qmax #xの中心らへんでyが一番高い部分に水を配置
    h[i,end] = 0 #一番上の水がある部分は、圧力変えた方がいい気がするけど、よくわらかない。
end



Sympyで出した$S_s$の式
$$
\displaystyle S_s = - \frac{\left(\alpha h\right)^{n} \left(Qmax - Qmin\right) \left(n - 1\right) \left(\left(\alpha h\right)^{n} + 1\right)^{- \frac{2 n - 1}{n}}}{h}
$$

In [109]:
function h_to_Q(h::Float64, Qmax::Float64, Qmin::Float64)
    alpha = 1.1　#適当。これも材質で違うんだろうなぁ。
    n = 3　#適当。これも材質で違うんだろうなぁ。
    Q = Qmin + (Qmax-Qmin)/(1+(alpha*h)^n)^(1-1/n)
    return Q
end



h_to_Q (generic function with 1 method)

In [110]:
function h_to_Ss(h::Float64, Qmax::Float64, Qmin::Float64)
    alpha = 1.1　#適当。これも材質で違うんだろうなぁ。
    n = 3　#適当。これも材質で違うんだろうなぁ。
    Q = h_to_Q(h,Qmax,Qmin)
    if Q == Qmax
        S_s = 0 #例外処理。水がセルからあふれてたら微分値はゼロとする。
    else
        S_s = -(((alpha*h)^n)*(Qmax-Qmin)*(n-1)*((alpha*h)^n+1)^(-(2*n-1)/n))/h
    end
    
    return S_s
end

h_to_Ss (generic function with 2 methods)

In [111]:
S_s = zeros(Lx,Ly)

for i in 1:Lx
    for j in 1:Ly
        S_s[i,j]=h_to_Ss(h[i,j],Qmax,Qmin)
    end
end

In [112]:
function ux_calc(h_i, h_i_hiku_1,k,dx)
    #uxは辺上でu[2,j]は実際は1+1/2でh[1,j]とh[2,j]の中間点。
    #なので-k(h[2,j]-h[1,j])/dxとなる。　kが辺上に点あり。
    ux = -k * (h_i - h_hiku_1)/dx
end

ux_calc (generic function with 1 method)

In [113]:
function uy_calc(h_i, h_i_hiku_1,k,dy)
    #uyは辺上でu[i,2]は実際は1+1/2でh[i,2]とh[i,1]の中間点。
    #なので-k((h[i,2]-h[i,1])/dx+1)となる。　kが辺上に点あり。
    ux = -k *((h_i - h_hiku_1)/dy + 1)
end

uy_calc (generic function with 1 method)

In [114]:
epsilon = 10^-5
TimeStep = 1
dt = 0.01
omega = 1.5

1.5

$$
h^{n+1}_{i,j} = \frac{b^{n+1}\Delta t +c^{n+1}\Delta t + d^{n+1/2}\Delta t + S_s h^n_{i,j}}{a^{n+1}\Delta t +S_s} \tag{8}
$$
$$
a^{n+1} = \frac{k_{x,i-1/2,j}+k_{x,i+1/2,j}}{\Delta x^2} + \frac{k_{y,i,j-1/2}+k_{y,i,j+1/2}}{\Delta y^2} \\
b^{n+1} = \frac{k_{x_{i-1/2,j}}h^{n+1}_{i-1,j}+k_{x_{i+1/2,j}}h^{n+1}_{i+1,j}}{\Delta x^2}\\
c^{n+1} = \frac{k_{y_{i,j-1/2}}h^{n+1}_{i,j-1}+k_{y_{i,j+1/2}}h^{n+1}_{i,j+1}}{\Delta y^2}\\
d^{n+1/2} = \frac{k_{y_{i,j+1/2}}-k_{y_{i,j-1/2}}}{\Delta y} \tag{9}
$$

In [115]:
using LinearAlgebra

In [116]:
timesteps = 0

while timesteps < TimeStep
    delta = 1
    n_iter = 0
    S_s_m = copy(S_s)
    h_n1_m = copy(h)
    h_n1_m1 = copy(h)
    
    while delta > epsilon
        
        for i in 1:Lx
            for j in 1:Ly
                if i==1 #境界条件。i-1とかu+1はiと等しいとする。
                    b = (k[i,j]*h_n1_m[i,j] + k[i+1,j]*h_n1_m[i+1,j])/dx^2
                elseif i == Ly
                    b = (k[i,j]*h_n1_m[i-1,j] + k[i+1,j]*h_n1_m[i,j])/dx^2
                else
                    b = (k[i,j]*h_n1_m[i-1,j] + k[i+1,j]*h_n1_m[i+1,j])/dx^2 
                end
                
                if j ==1 
                    c = (k[i,j]*h_n1_m[i,j] + k[i,j+1]*h_n1_m[i,j+1])/dy^2
                elseif j==Ly
                    c = (k[i,j]*h_n1_m[i,j-1] + k[i,j+1]*h_n1_m[i,j])/dy^2
                else                        
                    c = (k[i,j]*h_n1_m[i,j-1] + k[i,j+1]*h_n1_m[i,j+1])/dy^2
                end
                
                a = (k[i,j] + k[i+1,j])/dx^2 + (k[i,j] + k[i,j+1])/dy^2 #kのi+1/2はi+1に相当。
                d = (k[i,j+1] - k[i,j])/dy
                
                h_n1_m1[i,j] = (b*dt + c*dt + d*dt + S_s[i,j]*h[i,j]) / (a*dt + S_s[i,j])
                
                h_n1_m1[i,j] = h_n1_m[i,j] + omega*(h_n1_m1[i,j] - h_n1_m[i,j])
                S_s_m[i,j]=h_to_Ss(h_n1_m1[i,j],Qmax,Qmin)
            end
        end
        
        delta = norm(h_n1_m1 .- h_n1_m)
        
        
        if n_iter%10 == 0
            println("n_iter",n_iter,"delta=",delta)
        end
        
        n_iter += 1
        
        if n_iter >200
            return h_n1_m1
        end
        
        h_n1_m = copy(h_n1_m1)
        
    end
    
    timesteps += 1
end
        
    

n_iter0delta=27.087625125648394


LoadError: DomainError with -5.566687413578056:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

In [107]:
h

100×100 Array{Float64,2}:
 10.0  10.0  10.0  10.0  10.0  10.0  …  10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0  …  10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0  …  10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0
 10.0  10.0  10.0  10.0  10.0  10.0     10.0  10.0  10.0  10.0  10.0  10.0

In [5]:
#http://www.cas.cmc.osaka-u.ac.jp/~paoon/Lectures/2018-7Semester-AppliedMath9/04-nlsolve/
using NLsolve
function nls(func, params...; ini = [0.0])
    if typeof(ini) <: Number
        r = nlsolve((vout,vin)->vout[1]=func(vin[1],params...), [ini])
        v = r.zero[1]
    else
        r = nlsolve((vout,vin)->vout .= func(vin,params...), ini)
        v = r.zero
    end
    return v, r.f_converged
end

┌ Info: Precompiling NLsolve [2774e3e8-f4cf-5e23-947b-6d7e65073b56]
└ @ Base loading.jl:1278


nls (generic function with 1 method)

In [45]:
function Q_to_h(Q::Float64, Qmax::Float64, Qmin::Float64)
    alpha = 1.1
    n = 3
    
    h = nls(h_to_Q, Qmax,Qmin, ini=1.0)
    return h[1]
end

Q_to_h (generic function with 1 method)

In [18]:
Q_to_h(0.0,Qmax,Qmin)

6565.672785111952

In [57]:
for i in 0.0:0.1:10.0
    println(h_to_S_s(i,Qmax,Qmin))
end

0
-0.013280526297061819
-0.05230840913778783
-0.11294447479435378
-0.18583423962007378
-0.2574666961300097
-0.3144638608261558
-0.3484765734412118
-0.35826850359744095
-0.3481578401546089
-0.32479370219497583
-0.29450275141333504
-0.2620340121993115
-0.23039775687479383
-0.20121493268451668
-0.1751700625249827
-0.1523886259074779
-0.132698751297466
-0.11579378996714404
-0.10132519821941897
-0.08895132948444186
-0.07836023429922999
-0.06927801135754082
-0.06146962012828717
-0.0547361029780965
-0.048910377721935384
-0.04385272574851609
-0.039446519681895054
-0.03559441890509746
-0.032215096723727875
-0.029240482357455824
-0.02661346581425516
-0.024286002343903913
-0.022217553590419842
-0.02037380827536814
-0.018725632834724056
-0.017248210192701022
-0.015920332001101076
-0.014723815904021451
-0.013643024653431141
-0.012664468265161051
-0.011776473977384402
-0.01096891167558255
-0.010232964793964712
-0.009560938594752657
-0.008946099249867785
-0.00838253837590364
-0.007865058661281715
-0.

In [56]:
for i in 0.0:0.1:10.0
    println(h_to_Q(i,Qmax,Qmin))
end

0.5
0.4995568248521113
0.4964818658742202
0.4883686467361151
0.47347939398917643
0.4512437531159127
0.4224799262858254
0.3891255837593711
0.35359836194648997
0.3181377801424649
0.2844080089468402
0.25340803230339903
0.225578057217145
0.20097192801434255
0.17941569703433824
0.16062373814926464
0.1442725862989006
0.13004282024833808
0.11763996471345495
0.10680286090215102
0.09730513578463362
0.08895321243495419
0.08158283680217382
0.07505518936758504
0.06925311908683357
0.06407773940843337
0.059445466691616614
0.05528550075307294
0.05153771030341773
0.0481508723529139
0.04508121282399807
0.04229119931540133
0.03974854286234126
0.0374253719114428
0.03529754775791887
0.03334409603648043
0.031546733424286134
0.02988947253175556
0.028358291104567397
0.026940854233006895
0.025626280356983472
0.02440494355136679
0.023268305949892494
0.022208775278003322
0.021219583366016512
0.020294682244962028
0.01942865502145318
0.01861663921168926
0.017854260609530832
0.01713757608666894
0.01646302398795979